In [1]:
import pandas as pd

Let's load the dataset and extract just the comments, rather than using the entire dataset..

In [2]:
dataset = pd.read_csv("../datasets/Dataset.csv", encoding='ISO-8859-1')
dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [3]:
dataset.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

The dataset does not have any proper column names, lets provide some useful labels..

In [4]:
dataset.columns = ["sentiment", "id", "date", "query", "user", "tweet"]
dataset.head()

,sentiment,id,date,query,user,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


The only neccessary features at this point of time are sentiment, and tweet features..

In [5]:
dataset = dataset.drop(columns=["id", "date", "query", "user"])

In [6]:
dataset = dataset[:50000]

Load the transformer Model: RoBerta

In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline
model_path = "siebert/sentiment-roberta-large-english"

sentiment_pipeline = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, device=0) # device=0 means use the first GPU



c:\Users\Sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now let's try and evaluate our model's performance on our dataset:

In [8]:
from sklearn.model_selection import train_test_split


tweets = dataset['tweet'].tolist()
sentiments = dataset['sentiment'].tolist()

X_train, X_test, y_train, y_test = train_test_split(tweets, sentiments, test_size=0.2, random_state=42)



In [9]:
print(sentiment_pipeline("well then"))

[{'label': 'NEGATIVE', 'score': 0.990432858467102}]


In [10]:

def predict_sentiment(texts):
    results = sentiment_pipeline(texts)
    # Convert the 'positive' and 'negative' labels to 1 and 0 respectively.
    
    return [1 if res['label'] == 'POSITIVE' else 0 for res in results]

y_pred = predict_sentiment(X_test)


In [11]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.69      0.81     10000
           1       0.00      0.00      0.00         0

    accuracy                           0.69     10000
   macro avg       0.50      0.34      0.41     10000
weighted avg       1.00      0.69      0.81     10000



c:\Users\Sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

Let's plot the confusion Matrix:

In [12]:
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred)


array([[6868, 3132],
       [   0,    0]], dtype=int64)

Our overall performance might have taken a hit due to the fact that the Test Data only has 2 labels, but our pipeline produces labels for Three classes i.e. Positive, Neutral and Negative. On the whole, I believe that RoBERT can still be a viable tool for this.